# Tweet Analysis

In [ ]:
'''
Option 1
file = 'results.csv'
df_empty = pd.DataFrame()
with open(file) as fl:
    chunk_iter = pd.read_csv(fl, chunksize = 100000)
    for chunk in chunk_iter:
        chunk = chunk[chunk['column1'] > 180]
        df_empty = pd.concat([df_empty,chunk])
        
Option2
full_data = pd.concat(TextFileReader, ignore_index=True)

Option 3
df=TextFileReader.get_chunk(3)
'''

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import json

In [ ]:
users = pd.read_json('data/users.json', lines=True)

In [ ]:
users = users.drop(columns=['contributors_enabled','default_profile',
       'default_profile_image','entities','follow_request_sent', 'following', 'geo_enabled', 'has_extended_profile', 'id_str','is_translation_enabled', 'is_translator', 'lang', 'notifications', 'profile_background_color',
       'profile_background_image_url', 'profile_background_image_url_https',
       'profile_background_tile', 'profile_banner_url', 'profile_image_url',
       'profile_image_url_https','profile_image_url_https', 'profile_link_color',
       'profile_sidebar_border_color', 'profile_sidebar_fill_color',
       'profile_text_color', 'profile_use_background_image', 'protected','translator_type', 'url',
       'utc_offset', 'verified'])

In [ ]:
users.to_csv('data/users_v2.csv', index=False)

In [ ]:
file = 'data/tweets.json'
df_empty = pd.DataFrame()
with open(file) as fl:
    chunk_iter = pd.read_json(fl, lines=True, chunksize = 100000)
    for chunk in chunk_iter:
        chunk = chunk.drop(columns=["contributors", "coordinates", "geo", "display_text_range","entities", 'id', 'id_str',"lang", "place", "possibly_sensitive", "source", "truncated", "in_reply_to_status_id","in_reply_to_status_id_str", "in_reply_to_user_id", "in_reply_to_user_id_str", "extended_entities", "quoted_status_id" ,"quoted_status_id_str"])
        df_empty = pd.concat([df_empty,chunk])

In [ ]:
df_empty.to_csv('data/tweets_v2.csv', index=False)

In [ ]:
df_empty.columns

In [ ]:
df_empty= df_empty.drop(columns=["favorited","withheld_copyright", "withheld_in_countries", "withheld_scope" ])

## End of Cleaning Data

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import json

In [8]:
tweets = pd.read_csv("data/tweets_v2.csv")

In [3]:
users = pd.read_csv("data/users_v2.csv")

In [4]:
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

In [5]:
pysqldf("SELECT * FROM tweets LIMIT 10;").head()

,created_at,favorite_count,favorited,in_reply_to_screen_name,is_quote_status,retweet_count,retweeted,screen_name,text,user_id
0,2008-08-04 17:28:51,0,0,None,0,0,0,JohnBoozman,On a conference call about a weekend trip to I...,5558312
1,2008-08-06 19:04:45,0,0,None,0,0,0,JohnBoozman,Being interviewed by KTHV,5558312
2,2008-08-06 20:35:36,0,0,None,0,0,0,JohnBoozman,Being interviewed by KARN on his Arkansas Worl...,5558312
3,2008-08-07 13:52:52,0,0,None,0,0,0,JohnBoozman,"On KWHN in Fort Smith, that's 1320 AM",5558312
4,2008-08-07 15:12:05,0,0,None,0,0,0,JohnBoozman,Attending a Military of the Purple Heart Cerem...,5558312


In [12]:
tweets.describe().T

,count,mean,std,min,25%,50%,75%,max
favorite_count,1243370.0,2.008517e+02,3.545405e+03,0.0,0.0,2.0,8.0,9.848320e+05
retweet_count,1243370.0,1.900628e+02,9.944392e+03,0.0,1.0,4.0,10.0,3.637896e+06
user_id,1243370.0,1.397405e+16,1.053383e+17,5558312.0,33750798.0,234022257.0,993153006.0,8.547151e+17


In [16]:
pysqldf("SELECT favorite_count, screen_name FROM tweets ORDER BY favorite_count DESC  LIMIT 50;").head(20)

,favorite_count,screen_name
0,984832,SenSanders
1,627475,realDonaldTrump
2,569283,realDonaldTrump
3,527319,SenSanders
4,392346,realDonaldTrump
5,347262,realDonaldTrump
6,315624,SenatorMenendez
7,291081,realDonaldTrump
8,289727,realDonaldTrump
9,269437,realDonaldTrump


In [17]:
pysqldf("SELECT retweet_count, screen_name FROM tweets ORDER BY retweet_count DESC  LIMIT 50;").head(20)

,retweet_count,screen_name
0,3637896,GovSandoval
1,3637893,RogerMarshallMD
2,3637893,RepKevinYoder
3,3637853,marcorubio
4,3637837,SenCortezMasto
5,3637828,RepKihuen
6,3447358,RepMarkTakano
7,3447355,RepAnnieKuster
8,883078,RepCicilline
9,883078,RepPerlmutter


In [20]:
# person most favorited
pysqldf("SELECT count(favorite_count), screen_name FROM tweets WHERE favorite_count>1000 GROUP BY screen_name  ORDER BY Count(favorite_count) DESC  LIMIT 50;").head(20)

,count(favorite_count),screen_name
0,3041,realDonaldTrump
1,2931,SenSanders
2,989,SenWarren
3,717,timkaine
4,611,marcorubio
5,532,SenSchumer
6,498,ChrisMurphyCT
7,487,NancyPelosi
8,487,CoryBooker
9,376,RepAdamSchiff


In [21]:
# person most rted
pysqldf("SELECT count(retweet_count), screen_name FROM tweets WHERE retweet_count>1000 GROUP BY screen_name  ORDER BY Count(retweet_count) DESC  LIMIT 50;").head(20)

,count(retweet_count),screen_name
0,3233,realDonaldTrump
1,2657,SenSanders
2,641,POTUS
3,613,SenWarren
4,448,CoryBooker
5,429,SenSchumer
6,400,timkaine
7,327,RepAdamSchiff
8,318,ChrisMurphyCT
9,290,NancyPelosi


In [22]:
pysqldf("SELECT count(in_reply_to_screen_name), screen_name FROM tweets GROUP BY screen_name  ORDER BY Count(in_reply_to_screen_name) DESC  LIMIT 50;").head(20)
# person who replies the most

,count(in_reply_to_screen_name),screen_name
0,3094,DanaRohrabacher
1,1444,RepJeffDuncan
2,1302,SenatorDurbin
3,1278,justinamash
4,1029,SenGillibrand
5,894,jaredpolis
6,849,CoryBooker
7,803,SenBobCasey
8,794,SenFeinstein
9,757,RepMcGovern


In [23]:
pysqldf("SELECT count(in_reply_to_screen_name), in_reply_to_screen_name FROM tweets GROUP BY in_reply_to_screen_name  ORDER BY Count(in_reply_to_screen_name) DESC  LIMIT 50;").head(20)
# person who is being replied to the most INFLUENTIAL

,count(in_reply_to_screen_name),in_reply_to_screen_name
0,1309,SenatorDurbin
1,1011,SenGillibrand
2,798,SenFeinstein
3,753,RepMcGovern
4,736,SenBobCasey
5,612,SenJeffMerkley
6,545,GovMalloyOffice
7,531,PattyMurray
8,513,RepAdamSchiff
9,439,RonWyden


In [24]:
# person writing most tweets
pysqldf("SELECT count(screen_name), screen_name FROM tweets GROUP BY screen_name  ORDER BY Count(screen_name) DESC  LIMIT 50;").head(20)


,count(screen_name),screen_name
0,3258,RepDonBeyer
1,3252,SenatorDurbin
2,3250,MassGovernor
3,3250,GovMattBevin
4,3249,onetoughnerd
5,3249,SenTomCotton
6,3249,SenRonJohnson
7,3249,GrahamBlog
8,3248,RepScottPeters
9,3247,GovMalloyOffice
